In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from datetime import datetime
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND, MIND_news
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [ ]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [5]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'
# config.reducer = 'first'

# config.signal_length = 10
config.scale = 'large'
config.impr_size = 1000
config.mode = 'dev'
# config.fast = True
# config.news = 21563

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
file_directory = manager.path + "MIND/MIND{}_{}/".format(manager.scale, manager.mode)
# manager.gather_same_user_impr()

a = MIND(manager, file_directory)
# b = MIND_news(manager)

[2021-10-28 02:47:09,231] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDlarge_dev/1000/behaviors.pkl
[2021-10-28 02:47:10,750] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDlarge_dev/news.pkl
[2021-10-28 02:47:11,514] INFO (utils.utils) deduplicating...


In [ ]:
# manager.construct_whole_dataset()

In [7]:
behaviors = pickle.load(open('data/cache/bert/MINDlarge_dev/1000/behaviors.pkl','rb'))

In [8]:
len(behaviors['imprs']), len(a)

(255991, 255991)

In [9]:
len(behaviors['uindexes'])

255990

In [11]:
behaviors['uindexes'][-1]

682781

In [12]:
l = [behaviors['imprs'][i][0] for i in range(255991)]

In [13]:
d = pd.DataFrame(l)

In [16]:
d[d.duplicated()]

,0
127187,127186


In [5]:
news = dict(sorted(news.items(), key=lambda x: x[1], reverse=True))

In [6]:
news

{21563: 29681,
 27154: 29556,
 41719: 22557,
 3721: 20443,
 32061: 20385,
 12660: 19738,
 56055: 19678,
 4048: 18410,
 59445: 18186,
 22433: 18148,
 43631: 17624,
 44765: 17263,
 33144: 17009,
 61793: 16005,
 13997: 15422,
 33607: 14633,
 25620: 14136,
 52753: 13803,
 60685: 12898,
 1106: 12766,
 34029: 12674,
 44644: 12212,
 39800: 12189,
 10957: 12084,
 51910: 12043,
 46020: 11916,
 18477: 11747,
 53101: 11546,
 20911: 11524,
 47563: 11522,
 46648: 11483,
 3740: 10884,
 63248: 10731,
 56208: 10716,
 52186: 10655,
 49069: 10599,
 41025: 10563,
 63227: 10418,
 56989: 10262,
 26003: 10214,
 71567: 10160,
 4324: 9797,
 70422: 9702,
 70582: 9504,
 2834: 9306,
 26405: 9281,
 2696: 9246,
 49843: 9139,
 14997: 9078,
 142: 9070,
 51533: 9059,
 23304: 8937,
 21953: 8886,
 47039: 8728,
 17413: 8726,
 51922: 8701,
 60627: 8621,
 70479: 8510,
 19650: 8500,
 11040: 8479,
 1640: 8425,
 26698: 8244,
 17309: 8179,
 41829: 8100,
 60302: 7974,
 40212: 7934,
 27917: 7875,
 4649: 7822,
 53298: 7784,
 707

In [ ]:
r = a[0]

In [ ]:
r['his_attn_mask'].shape

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDlarge_test/bm25.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

index = 37405
news = t.convert_ids_to_tokens(a.encoded_news_original[index])
mask = a.attn_mask_original[index]
subword = a.subwords_original[index]

for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
# bm25, entity, first
index = 1

# news = t.convert_ids_to_tokens(a[index]['his_encoded_index'][1])
# mask = a[index]['his_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
# subword = a[index]['his_subword_index'][1]

# for i,j,k in zip(news,mask,subword):
#     print(i,j,k)
res = a[index]
ids = res['cdd_id'][1]
news = t.convert_ids_to_tokens(res['cdd_encoded_index'][1])
mask = res['cdd_attn_mask'][1]
# word = manager.convert_tokens_to_words(news)
subword = res['cdd_subword_index'][1]
print(ids)
for i,j,k in zip(news,mask,subword):
    print(i,j,k)

In [ ]:
news = t.convert_ids_to_tokens(a[0]['his_encoded_index'][1])
mask = a[0]['his_attn_mask'][1]
word = manager.convert_tokens_to_words(news)
subword = a[0]['his_subword_index'][1]
dedup = a[0]['his_refined_mask'][1]

for i,j,k,z in zip(news,mask,subword,dedup):
    print(i,j,k,z)